In [2]:
import os
import sys
import cv2
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
#import torch 
#from pykeops.torch import LazyTensor
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
from matplotlib import colors
import pickle

In [4]:
# experiment = "split_20231106"
experiment = "split_20231212"
%cd /home/plunder/.julia/dev/MechanochemicalPatterns/input/cell_tracking
# data = "RAF005"

/home/plunder/.julia/dev/MechanochemicalPatterns/input/cell_tracking


In [5]:
def open_df(data,experiment=experiment):
    df = pd.read_csv(experiment + "/Automation_out/"+data+"_spots.csv",encoding='cp1252',low_memory=False)
    df = df[3:]
    df = df.astype({
            "ID" : int,
            "TRACK_ID" : int, 
            "POSITION_X" : float,
            "POSITION_Y" : float,
            "POSITION_Z" : float,
            "POSITION_T" : float,
            "FRAME" : int,
            "MIN_INTENSITY_CH1" : float,
            "MAX_INTENSITY_CH1" : float,
            "MEDIAN_INTENSITY_CH1" : float,
            "TOTAL_INTENSITY_CH1" : float,
            "STD_INTENSITY_CH1" : float,
            "CONTRAST_CH1" : float,
            "SNR_CH1" : float
            })
    return df

def get_x_data(df,T,zmin=0.0,zmax=100.0):
    return df[(df["FRAME"]==T) & (df["POSITION_Z"]>=zmin) & (df["POSITION_Z"]<=zmax)]["POSITION_X"].to_numpy()
def get_y_data(df,T,zmin=0.0,zmax=100.0):
    return df[(df["FRAME"]==T) & (df["POSITION_Z"]>=zmin) & (df["POSITION_Z"]<=zmax)]["POSITION_Y"].to_numpy()
def get_snr(df,T,zmin=0.0,zmax=100.0):
    return df[(df["FRAME"]==T) & (df["POSITION_Z"]>=zmin) & (df["POSITION_Z"]<=zmax)]["SNR_CH1"].to_numpy()
def get_intensity(df,T,zmin=0.0,zmax=100.0):
    log_intensity = np.log10(df[(df["FRAME"]==T) & (df["POSITION_Z"]>=zmin) & (df["POSITION_Z"]<=zmax)]["TOTAL_INTENSITY_CH1"].to_numpy())
    log_intensity -= log_intensity.min()
    log_intensity *= 0.9/log_intensity.max()
    log_intensity += 0.09
    return log_intensity


def cmap(cell_type):
    if cell_type=="Proximal":
        # cmap_base = pl.cm.Purples
        cmap_base = colors.LinearSegmentedColormap.from_list("magenta", ["m","m"], N=1000)
    elif cell_type=="Distal":
        # cmap_base = pl.cm.Greens
        cmap_base = colors.LinearSegmentedColormap.from_list("magenta", ["g","g"], N=1000)
    else:
        cmap_base = pl.cm.Blues
    # Get the colormap colors
    cmap = cmap_base(np.arange(cmap_base.N))
    # Set alpha
    cmap[:,-1] = np.linspace(0, 1, cmap_base.N)**2
    # Create new colormap
    cmap = ListedColormap(cmap)
    return cmap

In [4]:
def sample_grid(n, dim=2,dtype=torch.float32,device="cpu"):
    grid_points = torch.linspace(0,1,n,dtype=dtype,device=device)
    grid_points = torch.stack(
        torch.meshgrid((grid_points,) * dim, indexing="ij"), dim=-1
    )
    grid_points = grid_points.reshape(-1, dim)
    return grid_points

def df_to_pos_normalized(df,T=1,zmin=0.0,zmax=100.0):
    x_data = get_x_data(df,T,zmin=zmin,zmax=zmax)
    y_data = get_y_data(df,T,zmin=zmin,zmax=zmax)
    x_data /= max(x_data.max(),y_data.max())
    y_data /= max(x_data.max(),y_data.max())
    return torch.tensor(np.c_[x_data,y_data],dtype=torch.float32)

def pos_grid_to_pixels(pos,grid,eps,intensity=None,scale=1.0):
    N = int(math.sqrt(len(grid)))
    x_j = LazyTensor(pos[None,:,:])
    y_i = LazyTensor(grid[:,None,:])
    K = (-((y_i - x_j)**2).sum(-1)/(2*eps**2)).exp()/(2*math.pi*eps**2)
    if intensity is None:
        grid_val = K.sum(1)/len(pos)
    else: 
        grid_val = K @ intensity
    return grid_val.reshape((N,N)) * scale

def df_to_smooth(data,N=512,eps=0.05,T=1,zmin=0.0,zmax=100.0,N0_prox=1140,N0_dist=1140):
    y_grid = sample_grid(N)
    df_prox = open_df(data + "_Proximal")
    df_dist = open_df(data + "_Distal")
    prox_data = df_to_pos_normalized(df_prox,T=T,zmin=zmin,zmax=zmax)
    dist_data = df_to_pos_normalized(df_dist,T=T,zmin=zmin,zmax=zmax)
    
    intensity_prox = torch.tensor(get_intensity(df_prox,T,zmin=zmin,zmax=zmax),dtype=torch.float32)
    intensity_prox /= intensity_prox.sum()
    intensity_dist = torch.tensor(get_intensity(df_dist,T,zmin=zmin,zmax=zmax),dtype=torch.float32)
    intensity_dist /= intensity_dist.sum()
    
    pixels_prox = pos_grid_to_pixels(prox_data,y_grid,eps,intensity_prox,scale=len(prox_data)/N0_prox)
    pixels_dist = pos_grid_to_pixels(dist_data,y_grid,eps,intensity_dist,scale=len(dist_data)/N0_dist)
    
    return pixels_prox, pixels_dist

In [5]:
def plot_smooth(pixels_prox,pixels_dist,alpha_prox=1.0,alpha_dist=1.0,vmin=0.0,vmax=2.0):
    fig_smooth, ax_smooth = plt.subplots()

    cmap_prox = cmap("Proximal")
    cmap_dist = cmap("Distal")

    im_prox = ax_smooth.imshow(pixels_prox.numpy().transpose(),origin='lower',cmap=cmap_prox,alpha=alpha_prox,vmin=vmin,vmax=vmax)
    im_dist = ax_smooth.imshow(pixels_dist.numpy().transpose(),origin='lower',cmap=cmap_dist,alpha=alpha_dist,vmin=vmin,vmax=vmax)
    ax_smooth.invert_yaxis()
    ax_smooth.set_aspect("equal")
    
    if alpha_prox>0.0:
        fig_smooth.colorbar(im_prox, ax=ax_smooth)
    if alpha_dist>0.0:
        fig_smooth.colorbar(im_dist, ax=ax_smooth)
    
    return fig_smooth, ax_smooth

In [6]:
def make_video(number_of_frames=None,prefix="t_",simu_name="simu",video_name='funny_video',frame_dir="frames",rate=40):
    img_array = []
    current_directory = os.getcwd()
    frame_directory = current_directory+"/"+simu_name+"/"+frame_dir
    if number_of_frames is None:
        number_of_frames = len([name for name in os.listdir(frame_directory) if os.path.isfile(os.path.join(frame_directory, name))])
    for count in range(number_of_frames):
        filename = frame_directory+"/"+prefix+str(count)+'.png'
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    out = cv2.VideoWriter(simu_name+"/"+video_name+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), rate,size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [7]:
def save_all(data):
    simu_name = experiment + "_" + data + "_smooth"
    os.mkdir(simu_name)
    os.mkdir(simu_name + "/Proximal")
    os.mkdir(simu_name + "/Distal")
    os.mkdir(simu_name + "/Proximal/frames")
    os.mkdir(simu_name + "/Proximal/data")
    os.mkdir(simu_name + "/Distal/frames")
    os.mkdir(simu_name + "/Distal/data")
    T = min(open_df(data + "_Proximal")["FRAME"].max(),open_df(data + "_Distal")["FRAME"].max())
    N0_prox = len(get_x_data(open_df(data + "_Proximal"),T,zmin=0.0,zmax=1000.0))
    N0_dist = len(get_x_data(open_df(data + "_Distal"),T,zmin=0.0,zmax=1000.0))
    prox, dist = df_to_smooth(data,T=T,zmin=0.0,zmax=1000.0,N0_prox=N0_prox,N0_dist=N0_dist)
    vmax_prox = prox.max()
    vmax_dist = dist.max()
    for t in tqdm(range(1,T+1)):
        pixels_prox, pixels_dist = df_to_smooth(data,T=t,zmin=0.0,zmax=1000.0,N0_prox=N0_prox,N0_dist=N0_dist)
        fig_prox, ax_prox = plot_smooth(pixels_prox, pixels_dist,alpha_dist=0.0,vmax=vmax_prox)
        fig_dist, ax_dist = plot_smooth(pixels_prox, pixels_dist,alpha_prox=0.0,vmax=vmax_dist)
        ax_prox.set_title(f"t={t}")
        ax_dist.set_title(f"t={t}")
        fig_prox.savefig(simu_name + "/Proximal/frames/t_"+str(t-1)+".png")
        fig_dist.savefig(simu_name + "/Distal/frames/t_"+str(t-1)+".png")
        plt.close()
        plt.close()
        with open(simu_name + "/Proximal/data/" + f"data_{t}.pkl",'wb') as file:
            pickle.dump(pixels_prox,file)
        with open(simu_name + "/Distal/data/" + f"data_{t}.pkl",'wb') as file:
            pickle.dump(pixels_dist,file)
    make_video(simu_name=simu_name+"/Proximal",video_name=simu_name+"_Proximal")
    make_video(simu_name=simu_name+"/Distal",video_name=simu_name+"_Distal")


In [8]:
# save_all(data)

In [9]:
# for k in range(1,10):
#     try:
#         save_all("FWF00" + str(k))
#     except:
#         print("There is a problem with FWF00" + str(k))
#     try:
#         save_all("NCF00" + str(k))
#     except:
#         print("There is a problem with NCF00" + str(k))
#     try:
#         save_all("RAF00" + str(k))
#     except:
#         print("There is a problem with RAF00" + str(k))

In [10]:
# %matplotlib widget
# data = "FWF003"
# N0_prox = len(get_x_data(open_df(data + "_Proximal"),100,zmin=0.0,zmax=1000.0))
# N0_dist = len(get_x_data(open_df(data + "_Distal"),100,zmin=0.0,zmax=1000.0))
# prox, dist = df_to_smooth(data,T=287,zmin=0.0,zmax=1000.0,N0_prox=N0_prox,N0_dist=N0_dist,eps=0.05)
# X = np.linspace(0,1,512)
# Y = np.linspace(0,1,512)
# X, Y = np.meshgrid(X, Y)
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# surf_prox = ax.plot_surface(X, Y, prox, cmap="Purples",linewidth=0, antialiased=False)
# surf_dist = ax.plot_surface(X, Y, dist, cmap="Greens",linewidth=0, antialiased=False)

In [11]:
# df = open_df("NCF004" + "_Proximal")

In [12]:
# open_df("NCF004" + "_Distal")

In [8]:
def replot(data):
    simu_name = experiment + "_" + data + "_smooth"
    os.mkdir(simu_name + "/Proximal/frames_replot")
    os.mkdir(simu_name + "/Distal/frames_replot")
    T = min(open_df(data + "_Proximal")["FRAME"].max(),open_df(data + "_Distal")["FRAME"].max())
    N0_prox = len(get_x_data(open_df(data + "_Proximal"),T,zmin=0.0,zmax=1000.0))
    N0_dist = len(get_x_data(open_df(data + "_Distal"),T,zmin=0.0,zmax=1000.0))
    with open(simu_name + "/Proximal/data/" + f"data_{T}.pkl",'rb') as file:
            pixels_prox = pickle.load(file)
    with open(simu_name + "/Distal/data/" + f"data_{T}.pkl",'rb') as file:
        pixels_dist = pickle.load(file)
    vmax_prox = pixels_prox.max()
    vmax_dist = pixels_dist.max()
    for t in tqdm(range(1,T+1)):
        with open(simu_name + "/Proximal/data/" + f"data_{t}.pkl",'rb') as file:
            pixels_prox = pickle.load(file)
        with open(simu_name + "/Distal/data/" + f"data_{t}.pkl",'rb') as file:
            pixels_dist = pickle.load(file)
        fig_prox, ax_prox = plot_smooth(pixels_prox, pixels_dist,alpha_dist=0.0,vmax=vmax_prox)
        fig_dist, ax_dist = plot_smooth(pixels_prox, pixels_dist,alpha_prox=0.0,vmax=vmax_dist)
        ax_prox.set_title(f"t={t}")
        ax_dist.set_title(f"t={t}")
        fig_prox.savefig(simu_name + "/Proximal/frames_replot/t_"+str(t-1)+".png")
        fig_dist.savefig(simu_name + "/Distal/frames_replot/t_"+str(t-1)+".png")
        plt.close()
        plt.close()
    make_video(simu_name=simu_name+"/Proximal",video_name=simu_name+"_Proximal_replot",frame_dir="frames_replot")
    make_video(simu_name=simu_name+"/Distal",video_name=simu_name+"_Distal_replot",frame_dir="frames_replot")


In [7]:
plt.style.use('dark_background')

replot("FWF003")

FileExistsError: [Errno 17] File exists: 'split_20231212_FWF003_smooth/Proximal/frames_replot'

In [15]:
plt.style.use('dark_background')

replot("FWF006")

100%|██████████| 290/290 [01:09<00:00,  4.17it/s]


In [16]:
# for k in range(1,10):
#     try:
#         replot("FWF00" + str(k))
#     except:
#         print("There is a problem with FWF00" + str(k))
#     try:
#         replot("NCF00" + str(k))
#     except:
#         print("There is a problem with NCF00" + str(k))
#     try:
#         replot("RAF00" + str(k))
#     except:
#         print("There is a problem with RAF00" + str(k))